# Calculating ROUGE scores
This notebook's purpose is to provide easy-to-install environment for calculating ROUGE scores on provided summaries.

## Prequisites
To calculate ROUGE score, one must connect to Google Drive and have these directories and files:
* directory `result/<model_name>` with zipped directory with summaries generated by the model after running `main.py` script and zipped directory with gold standard summaries.

## Mounting Google Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Installing ROUGE and Python wrapper - `pyrouge`

In [ ]:
!git clone https://github.com/andersjo/pyrouge.git rouge

In [ ]:
!pip install pyrouge

In [4]:
!pyrouge_set_rouge_path '/content/rouge/tools/ROUGE-1.5.5'

2022-05-08 11:59:28,725 [MainThread  ] [INFO ]  Set ROUGE home directory to /content/rouge/tools/ROUGE-1.5.5.


In [ ]:
%cd rouge/tools/ROUGE-1.5.5/data
!rm "WordNet-2.0.exc.db"
!perl ./WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db
!cpan install XML::DOM

In [8]:
from pyrouge import Rouge155
import csv
import os

## Testing on MatchSum summaries

In [ ]:
r = Rouge155()
r.system_dir = '/content/drive/MyDrive/result/MatchSum_cnndm_bert.ckpt/dec/'
r.model_dir = '/content/drive/MyDrive/result/MatchSum_cnndm_bert.ckpt/ref/'
r.system_filename_pattern = '(\d+).dec'
r.model_filename_pattern = '#ID#.ref'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

In [ ]:
with open("/content/drive/MyDrive/result/MatchSum_cnndm_bert.ckpt/rouge.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["metric", "value"])
    writer.writeheader()
    for metric, value in output_dict.items():
        writer.writerow({"metric": metric, "value":value})

# Our models results

In [ ]:
%env SUMMARY_DIR=/content/drive/MyDrive/result/pso_first_model/
%env SUMMARY_FILE=summary.zip
%env TARGET_DIR=/content/drive/MyDrive/result/
%env TARGET_FILE=targets.zip

In [ ]:
! unzip $SUMMARY_DIR$SUMMARY_FILE -d $SUMMARY_DIR

In [ ]:
! unzip $TARGET_DIR$TARGET_FILE -d $TARGET_DIR

In [ ]:
r = Rouge155()
r.system_dir = os.environ["SUMMARY_DIR"] + os.environ["SUMMARY_FILE"].split(".")[0]
r.model_dir = os.environ["TARGET_DIR"] + os.environ["TARGET_FILE"].split(".")[0]
r.system_filename_pattern = '(\d+).txt'
r.model_filename_pattern = '#ID#.txt'

output = r.convert_and_evaluate()
output_dict = r.output_to_dict(output)

In [14]:
with open(os.environ["SUMMARY_DIR"] + "rouge.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["metric", "value"])
    writer.writeheader()
    for metric, value in output_dict.items():
        writer.writerow({"metric": metric, "value":value})